# Imports

In [4]:
from sklearn.cluster import KMeans
import os
import pandas as pd
from obp.dataset import OpenBanditDataset
from obp.ope import OffPolicyEvaluation, InverseProbabilityWeighting as IPW
from obp.policy import BernoulliTS
from obp.ope import (
    OffPolicyEvaluation, 
    RegressionModel,
    DirectMethod,
    InverseProbabilityWeighting,
    DoublyRobust
)
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import matplotlib as plt
import time
import pickle


---------------------------------------------------------------------------------------------------------------------------------------

# Versión automática

In [5]:
def explorar_clusters(n_clusters, particion):

    # Importar dataset original desde una copia: 'men_copy.csv'
    path = os.path.join("open_bandit_dataset", "random", "all", "all_copy.csv")
    dataset = pd.read_csv(path,  index_col=0) # por ahora solo se utilizan datos 'men', 'random'

    print("Tamaño del dataset completo: ", dataset.shape)

    full_dataset = dataset.copy()

    # user_features = dataset[["user_feature_0", "user_feature_1", "user_feature_2", "user_feature_3"]]
    # user_features_encoded = pd.get_dummies(user_features, columns=["user_feature_0", "user_feature_1", "user_feature_2", "user_feature_3"])

    user_features_encoded =  dataset.drop(columns=["item_id", "position", "click", "propensity_score", "timestamp", "user_feature_0", "user_feature_1", "user_feature_2", "user_feature_3"])

    kmeans = KMeans(init="k-means++", n_clusters=n_clusters, random_state=12345)
    predict = kmeans.fit_predict(user_features_encoded)

    full_dataset['clusters'] = predict

    cluster_datasets = []
    cluster_results = []
    cluster_sizes = []
    bandit_dataset = []
    # Aplicar una mascara al dataset para filtrar por cluster
    for i in range(n_clusters): 
        print(i)
        cluster_datasets.append(full_dataset.loc[full_dataset['clusters'] == i])

    # Iteracion principal para sobreescribir csv de OBP
    for i in range(n_clusters):

        ################# potencial problema ###############
        path = os.path.join("open_bandit_dataset", "random", "all", "all.csv")
        dataset = cluster_datasets[i]
        cluster_size = dataset.shape[0]

        dataset.to_csv(path, index=False)
        print("Tamaño del dataser de cluster ", i, ": " , dataset.shape)
        ####################################################

        dataset = OpenBanditDataset(behavior_policy="random", campaign="all", data_path="open_bandit_dataset", dataset_name="all.csv")
        bandit_feedback = dataset.obtain_batch_bandit_feedback()

        bandit_dataset.append(dataset)
        
        ######### Aqui aplicar simulacion y estimacion sobre clusters ###########
        evaluation_policy = BernoulliTS(
            n_actions=dataset.n_actions, 
            len_list=dataset.len_list, 
            campaign="all",
            random_state=12345,
            policy_name = "random"
        )

        regression_model = RegressionModel(
            n_actions=dataset.n_actions,
            len_list=dataset.len_list,
            action_context=dataset.action_context,
            base_model=LogisticRegression(max_iter=1000, random_state=12345),
        )

        estimated_rewards_by_reg_model = regression_model.fit_predict(
            context=bandit_feedback["context"],
            action=bandit_feedback["action"],
            reward=bandit_feedback["reward"],
            position=bandit_feedback["position"],
            pscore=bandit_feedback["pscore"],
            n_folds=3, # use 3-fold cross-fitting
            random_state=12345,
        )
        
        action_dist = evaluation_policy.compute_batch_action_dist(
            n_sim=100000, n_rounds=bandit_feedback["n_rounds"],
        )

        # estimate the policy value of BernoulliTS based on its action choice probabilities
    # it is possible to set multiple OPE estimators to the `ope_estimators` argument
        ope = OffPolicyEvaluation(
            bandit_feedback=bandit_feedback,
            ope_estimators=[InverseProbabilityWeighting(), DirectMethod(), DoublyRobust()]
        )
        

        # `summarize_off_policy_estimates` returns pandas dataframes including the OPE results
        estimated_policy_value, estimated_interval = ope.summarize_off_policy_estimates(
            action_dist=action_dist, 
            estimated_rewards_by_reg_model=estimated_rewards_by_reg_model,
            n_bootstrap_samples=10000, # number of resampling performed in bootstrap sampling.
            random_state=12345,
        )

        cluster_results.append(estimated_policy_value)
        cluster_sizes.append(cluster_size)
    return cluster_results, cluster_sizes


In [ ]:
cluster_sizes = [2, 3, 4, 5, 10, 15]

cluster_results = {}       # resultados OPE
cluster_size_results = {}  # tamanos de cada cluster evaluado

for cluster_size in cluster_sizes: 
    print(f"\n---------------------- Explorando con {cluster_size} cluster(s) -----------------------\n")
    try:

        experiment_results, experiment_size_results = explorar_clusters(n_clusters=cluster_size, particion='all')
        cluster_results[cluster_size] = experiment_results
        cluster_size_results[cluster_size] = experiment_size_results
        print(cluster_size_results)
    except: 
        print("Ejecución detenida en cluster_size: ", cluster_size)
        break
print(f"\n------------------------------ Resultados ------------------------------\n")
print(cluster_results)

# Para guardar los resultados de estimadores
with open('results_all_user_affinity.pkl', 'wb') as f:
    pickle.dump(cluster_results, f)

# Para guardar los tamaños de los clusters    
with open('size_results_all_user_affinity.pkl', 'wb') as f: 
    pickle.dump(cluster_size_results, f)



---------------------- Explorando con 2 cluster(s) -----------------------

Tamaño del dataset completo:  (1374327, 89)
0
1
Tamaño del dataser de cluster  0 :  (1372115, 90)
Tamaño del dataser de cluster  1 :  (2212, 90)
{2: [1372115, 2212]}

---------------------- Explorando con 3 cluster(s) -----------------------

Tamaño del dataset completo:  (1374327, 89)
0
1
2
Tamaño del dataser de cluster  0 :  (1367927, 90)
Tamaño del dataser de cluster  1 :  (2212, 90)
Tamaño del dataser de cluster  2 :  (4188, 90)
{2: [1372115, 2212], 3: [1367927, 2212, 4188]}

---------------------- Explorando con 4 cluster(s) -----------------------

Tamaño del dataset completo:  (1374327, 89)
0
1
2
3
Tamaño del dataser de cluster  0 :  (1363520, 90)
Tamaño del dataser de cluster  1 :  (2212, 90)
Tamaño del dataser de cluster  2 :  (4185, 90)
Tamaño del dataser de cluster  3 :  (4410, 90)
{2: [1372115, 2212], 3: [1367927, 2212, 4188], 4: [1363520, 2212, 4185, 4410]}

---------------------- Explorando con 5